<a href="https://colab.research.google.com/github/bsun1220/UCBTradingComp2022/blob/main/UCBerkeley2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import glob
import zipfile
import functools

In [4]:
import os

# Upload the API token.
def get_kaggle_credentials():
  token_dir = os.path.join(os.path.expanduser("~"),".kaggle")
  token_file = os.path.join(token_dir, "kaggle.json")
  if not os.path.isdir(token_dir):
    os.mkdir(token_dir)
  try:
    with open(token_file,'r') as f:
      pass
  except IOError as no_file:
    try:
      from google.colab import files
    except ImportError:
      raise no_file
  uploaded = files.upload()
  if "kaggle.json" not in uploaded:
      raise ValueError("You need an API key! see: "
          "https://github.com/Kaggle/kaggle-api#api-credentials")
  with open(token_file, "wb") as f:
    f.write(uploaded["kaggle.json"])
  os.chmod(token_file, 600)

get_kaggle_credentials()

Saving kaggle.json to kaggle.json


In [6]:
!kaggle competitions download -c berkeleytradingcompetition2022

 79% 9.00M/11.4M [00:00<00:00, 27.9MB/s]
100% 11.4M/11.4M [00:00<00:00, 32.6MB/s]
  0% 0.00/3.31M [00:00<?, ?B/s]
100% 3.31M/3.31M [00:00<00:00, 185MB/s]


In [7]:
!unzip test.csv.zip
!unzip train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")